In [1]:
import math
import random
import operator
import pandas as pd

In [2]:
from collections import defaultdict

In [3]:
import numpy as np

Action1项目要求:    
针对Delicious数据集，对SimpleTagBased算法进行改进（使用NormTagBased、TagBased-TFIDF算法）

# 1 加载数据集

In [4]:
df = pd.read_csv('./data/user_taggedbookmarks-timestamps.dat', sep='\t')

## 1.1 认识数据集
---
用户user对bookmark书签进行打标签tag的数据集。    

网站的tag可以理解为分类；     

用户的tag可以理解为兴趣；

In [5]:
df.head(2)

,userID,bookmarkID,tagID,timestamp
0,8,1,1,1289255362000
1,8,2,1,1289255159000


In [6]:
# 一共437593个数据，具有4个特征
df.shape

(437593, 4)

In [7]:
df.isnull().any()  # 检查每一列是否有空值

userID        False
bookmarkID    False
tagID         False
timestamp     False
dtype: bool

In [8]:
# 一共有1867个用户
df['userID'].unique().shape

(1867,)

In [9]:
# 去重后一共有40897个标签
df['tagID'].unique().shape

(40897,)

In [10]:
# 去重后一共有69223个书签
df['bookmarkID'].unique().shape

(69223,)

## 1.2构建`{userID:{bookmarkID:[tagIDs]}}`映射关系

In [11]:
# 字典类型，保存user对item的tag，即:{userid: {item1:[tag1, tag2], ...}}
records = {}

# 训练集，测试集
train_data = dict()
test_data = dict()

# {用户u:{标签t:用户u使用过标签t的次数}}
user_tags = dict()

# {标签t:{商品i:标签t打在商品i上的次数}}
tag_items = dict()

# {用户u:{商品i:用户u使用商品i的次数}}
user_items = dict()

# {标签t: {用户u: 标签t被用户u使用的次数}}
tag_users = {}

# {商品i: {用户u: 商品i被用户u打过标签的次数}}
item_users = {}

# {商品i: {标签t: 商品i被打过标签t的次数}}
item_tags = {}

`dict.setdefault(key, default=None)`
如果键不存在于字典中，将会添加键并将值设为默认值。

In [12]:
%%time
for i in range(df.shape[0]):
    uid = df['userID'][i]
    iid = df['bookmarkID'][i]
    tag = df['tagID'][i]
    # 键不存在时，新增键，且设置value为{}
    records.setdefault(uid,{})
    records[uid].setdefault(iid,[])
    records[uid][iid].append(tag)
    
print("数据集大小为 %d." % (len(df)))
print("设置tag的人数 %d." % (len(records)))

数据集大小为 437593.
设置tag的人数 1867.
CPU times: user 19.3 s, sys: 52 ms, total: 19.3 s
Wall time: 19.3 s


In [13]:
# userID=8的这个用户,给书签79所打的标签
records[8][79]

[45, 88, 89, 90, 91, 92, 93]

# 2 拆分数据集
每个用户内的书签中取一部分作为测试集

python中的元素有可变和不可变之分，如整数、浮点数、字符串、元组等都属于不可变的，而字典和列表属于可变的。     

字典和列表的可变指的是它们自身可以增加和删除元素，或者修改元素的值。    

在函数调用时，若提供的是不可变对象，那么在函数内部对其修改时，不会影响函数外部的值，但若提供的是可变对象，则在函数内部对它修改时，在函数外部的值也会被改变。    

因此下面`test_data`和`train_data`虽然未给函数传参，但仍然会修改函数外部的`test_data`和`train_data`。

In [14]:
# 将数据集拆分为训练集和测试集
def train_test_split(ratio, seed=100):
    random.seed(seed)
    m, n = 0, 0
    # u 是每个用户id
    for u in records.keys():
    	# i 是每个用户收藏的书签ID
        for i in records[u].keys():
            # ratio比例设置为测试集
            if random.random() < ratio:
                test_data.setdefault(u, {})
                test_data[u].setdefault(i, [])
                # t 是每个书签被打的标签id
                for t in records[u][i]:
                    test_data[u][i].append(t)
                    n += 1
            else:
                train_data.setdefault(u, {})
                train_data[u].setdefault(i, [])
                for t in records[u][i]:
                    train_data[u][i].append(t)  
                    m += 1      
    print("训练集样本数 %d, 测试集样本数 %d" % (len(train_data), len(test_data)))
    print("测试集总标签数:%d" % n)
    print("训练集总标签数:%d" % m)
    print("测试集总标签数占总样本的{:.2f}%".format(n / (m + n) * 100))

In [15]:
%%time
train_test_split(ratio=0.2)

训练集样本数 1860, 测试集样本数 1793
测试集总标签数:86772
训练集总标签数:350821
测试集总标签数占总样本的19.83%
CPU times: user 242 ms, sys: 17 ms, total: 259 ms
Wall time: 258 ms


当设置ratio=0.2时，最终拆分到的测试集标签数占总标签数的19.83%；当ratio=0.3时，拆分到的测试集占比也刚好趋近于30%。     
其中的原理是蒙特卡洛模拟，当重复的次数越多，小于0.2的随机产生的数所占比例就越趋近于0.2，因为条件是`random.random()<ratio`, 因此测试集大小趋近于0.2，但不等于0.2。

In [16]:
# 与函数内打印的结果一致
# 说明函数内部在调用函数外的字典的同时，对它的修改也会在函数外生效，因为字典是不可变容器
len(train_data), len(test_data)

(1860, 1793)

# 3 构建用户与标签、标签与商品、用户与商品的映射关系
即：初始化user_tags, tag_items, user_items、tag_users、item_users字典

In [17]:
# 设置字典 mat{index: {item: 1}
def addValueToMat(mat, index, item, value=1):
    # 假如index在mat字典中不存在，则新建该键
    if index not in mat:
        mat.setdefault(index,{})
        mat[index].setdefault(item, value)
    else:
        if item not in mat[index]:
            mat[index][item] = value
        else:
            mat[index][item] += value


# 使用训练集，初始化user_tags, tag_items, user_items
def initStat():
    records=train_data
    # u是用户id, items是书签id的字典
    for u, items in records.items():
    	# i是书签id，tags是标签的列表
        for i, tags in items.items():
        	# tag是标签id
            for tag in tags:
                # 用户和tag的关系
                addValueToMat(user_tags, u, tag, 1)
                # tag和item的关系
                addValueToMat(tag_items, tag, i, 1)
                # 用户和item的关系
                addValueToMat(user_items, u, i, 1)
                # tag和用户的关系
                addValueToMat(tag_users, tag, u, 1)
                # item和用户的关系
                addValueToMat(item_users, i, u, 1)
                # item和tag关系
                addValueToMat(item_tags, i, tag, 1)
    print("user_tags, tag_items, user_items初始化完成.")
    print("len(user_tags): %d, len(tag_items): %d, len(user_items): %d" % \
          (len(user_tags), len(tag_items), len(user_items)))
    print("len(tag_users):", len(tag_users))
    print("len(item_users)", len(item_users))

In [18]:
%%time 
initStat()

user_tags, tag_items, user_items初始化完成.
len(user_tags): 1860, len(tag_items): 36884, len(user_items): 1860
len(tag_users): 36884
len(item_users) 59555
CPU times: user 1.25 s, sys: 66 ms, total: 1.31 s
Wall time: 1.31 s


# 4对指定userID推荐Top-N

## 4.1 SimpleTagBased算法

用户u对商品i的兴趣：
$$score(u, i) = \sum_{t}user\_tags[u, t] * tag\_items[t, i]$$
- 其中：
    - $user\_tags[u, t]$ 表示用户u使用过标签t的次数；    
    - $tag\_items[t, i]$ 表示商品i被打过标签t的次数；

对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和：$\sum(wut * wti)$

### 4.1.1 定义算法函数

In [19]:
# 对用户user推荐Top-N
def recommend(user, N):
    # {item: score}
    recommend_items = dict()
    # tagged_items的同一个userid下的字典，键是书签id，值是用户给该书签打的标签数量
    tagged_items = user_items[user] 
    # tag 是标签id，wut是用户对某标签使用的次数
    for tag, wut in user_tags[user].items():
        # item是商品 即：书签id，wti是该商品i被打上t标签的次数 
        for item, wti in tag_items[tag].items():
            if item in tagged_items:
                continue
            if item not in recommend_items:
                #  用户u对商品i的兴趣
                recommend_items[item] = wut * wti
            else:
                recommend_items[item] += wut * wti
    return sorted(recommend_items.items(), 
                  key = operator.itemgetter(1), # 等价于lambda x:x[1]
                  reverse=True)[0:N]  # 返回top-N个商品和用户对它的兴趣分 

In [20]:
%%time
user = 8
N = 3
# 为用户id 8 推荐他可能感兴趣的top3商品(书签id:兴趣分)
recommend(user, N)

CPU times: user 11.2 ms, sys: 2.01 ms, total: 13.2 ms
Wall time: 13 ms


[(1416, 61), (1526, 50), (4535, 47)]

### 4.1.2 评估SimpleTagBased算法

对于用户u，令R(u)为给用户u的长度为N的推荐列表，里面包含我们认为用户会打标签的物品。令T(u)是测试集中用户u实际上打过标签的物品集合。     

然后，我们利用准确率( precision)和召回率( recall)评测个性化推荐算法的精度。

- 精确率：表示推荐给用户的物品中，用户确实打过标签的物品占比；    

$$Precision = \frac{|R(u)\bigcap T(u)|}{|R(u)|}$$      

- 召回率：表示在用户实际打过标签的所有物品中，被推荐给用户的物品占比；    

$$Recall = \frac{|R(u)\bigcap T(u)|}{|T(u)|}$$

In [21]:
# 使用测试集，计算精确率和召回率
def precisionAndRecall(N):
    hit = 0
    h_recall = 0
    h_precision = 0
    # user用户，items是字典{书签ids: tag列表}
    for user, items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        rank = recommend(user, N)
        # item商品(书签ID)，rui是兴趣分
        for item, rui in rank:
            # 如果推荐的商品在该用户的书签字典中，说明推荐对了，则hit+1
            if item in items:
                hit = hit + 1
        # len(items) 实际打过标签的物品数
        h_recall += len(items)
        h_precision += N
    # 返回精确率 和 召回率
    prec = hit / (h_precision * 1.0)
    rec = hit / (h_recall * 1.0)
    return prec, rec


# 使用测试集，对推荐结果进行评估
def testRecommend():
    print("推荐结果评估")
    # %4s 其中s前的4表示占位4个字符
    print("%3s %10s %10s" % ('N',"精确率",'召回率'))
    for n in [5, 10, 20, 40, 60, 80, 100]:
        precision, recall = precisionAndRecall(n)
        print("%3d  %10.3f%%  %10.3f%%" % (n, precision * 100, recall * 100))

In [22]:
%%time
print("SimpleTagBased算法:")
testRecommend()

SimpleTagBased算法:
推荐结果评估
  N        精确率        召回率
  5       0.829%       0.355%
 10       0.633%       0.542%
 20       0.512%       0.877%
 40       0.381%       1.304%
 60       0.318%       1.635%
 80       0.276%       1.893%
100       0.248%       2.124%
CPU times: user 3min 5s, sys: 1.87 s, total: 3min 7s
Wall time: 3min 7s


## 4.2 NormTagBased-2算法推荐
对score进行归一化

用户u对商品i的兴趣：
$$score(u, i) = \sum_{t}\frac{user\_tags[u, t]}{user\_tags[u]} * \frac{tag\_items[t, i]}{tag\_items[t]}$$
- 其中：
    - $user\_tags[u, t]$ 表示用户u使用过标签t的次数；    
    - $tag\_items[t, i]$ 表示商品i被打过标签t的次数；
    - $user\_tags[u]$ 表示用户u打过多少个标签；
    - $tag\_items[t]$ 表示被打过标签t的商品一共多少个。

### 4.2.1 定义算法函数

In [23]:
def recommend_by_norm_2(user, N):
    # 先找到用户打过标签的商品{item:[tag...]}
    tagged_items = user_items[user]
    # 创建推荐字典，存储推荐商品和兴趣分
    recommend_items = defaultdict(int)
    # 用户u用过的标签t, 以及他使用标签t的次数wut
    for t, wut in user_tags[user].items():
        # 被打过标签t的商品i，以及商品i被打上标签t的次数wti
        for i, wti in tag_items[t].items():
            # 目的是推荐用户没打过标签的，但又符合他兴趣的商品
            # 因此遇到用户已打过标签的商品就跳过
            if i in tagged_items:
                continue
            recommend_items[i] += (wut / len(user_tags[user])) * (wti / len(tag_items[t]))
    return sorted(recommend_items.items(), 
                 key=operator.itemgetter(1),
                 reverse=True)[0:N]

In [24]:
%%time
recommend_by_norm_2(8, 3)

CPU times: user 21 ms, sys: 2 ms, total: 23 ms
Wall time: 22.5 ms


[(15, 0.007202754900925632),
 (23702, 0.006280946268109297),
 (66188, 0.00627448884327588)]

### 4.2.2 评估NormTagBased-2算法

In [25]:
# 使用测试集，计算精确率和召回率
def precisionAndRecall(N):
    hit = 0
    h_recall = 0
    h_precision = 0
    # user用户，items是字典{书签ids: tag列表}
    for user, items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        rank = recommend_by_norm_2(user, N)
        # item商品(书签ID)，rui是兴趣分
        for item, rui in rank:
            # 如果推荐的商品在该用户的书签字典中，说明推荐对了，则hit+1
            if item in items:
                hit = hit + 1
        # len(items) 实际打过标签的物品数
        h_recall += len(items)
        h_precision += N
    # 返回精确率 和 召回率
    prec = hit / (h_precision * 1.0)
    rec = hit / (h_recall * 1.0)
    return prec, rec

In [26]:
%%time
print("NormTagBased-2算法:")
testRecommend()

NormTagBased-2算法:
推荐结果评估
  N        精确率        召回率
  5       0.806%       0.345%
 10       0.577%       0.494%
 20       0.428%       0.733%
 40       0.300%       1.026%
 60       0.259%       1.333%
 80       0.237%       1.620%
100       0.222%       1.903%
CPU times: user 6min 5s, sys: 1.41 s, total: 6min 6s
Wall time: 6min 6s


## 4.3 NormTagBased-1算法推荐

用户u对商品i的兴趣：
$$score(u, i) = \sum_{t}\frac{user\_tags[u, t]}{user\_tags[u]} * \frac{tag\_items[t, i]}{tag\_users[t]}$$
- 其中：
    - $user\_tags[u, t]$ 表示用户u使用过标签t的次数；    
    - $tag\_items[t, i]$ 表示商品i被打过标签t的次数；
    - $user\_tags[u]$ 表示用户u打过多少个标签；
    - $tag\_users[t]$ 表示打过标签t的用户数。

### 4.3.1 自定义推荐函数

In [27]:
def recommend_by_norm_1(user, N):
    # 先找到用户打过标签的商品{item:[tag...]}
    tagged_items = user_items[user]
    # 创建推荐字典，存储推荐商品和兴趣分
    recommend_items = defaultdict(int)
    # 用户u用过的标签t, 以及他使用标签t的次数wut
    for t, wut in user_tags[user].items():
        # 被打过标签t的商品i，以及商品i被打上标签t的次数wti
        for i, wti in tag_items[t].items():
            # 目的是推荐用户没打过标签的，但又符合他兴趣的商品
            # 因此遇到用户已打过标签的商品就跳过
            if i in tagged_items:
                continue
            recommend_items[i] += (wut / len(user_tags[user])) * (wti / len(tag_users[t]))
    return sorted(recommend_items.items(), 
                 key=operator.itemgetter(1),
                 reverse=True)[0:N]

In [28]:
%%time
recommend_by_norm_1(8, 3)

CPU times: user 25.7 ms, sys: 998 µs, total: 26.7 ms
Wall time: 26.1 ms


[(23702, 0.010806443468161389),
 (66188, 0.010794779741906341),
 (28750, 0.010478474479903197)]

### 4.3.2 评估NormTagBased-1算法

In [29]:
# 使用测试集，计算精确率和召回率
def precisionAndRecall(N):
    hit = 0
    h_recall = 0
    h_precision = 0
    # user用户，items是字典{书签ids: tag列表}
    for user, items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        rank = recommend_by_norm_1(user, N)
        # item商品(书签ID)，rui是兴趣分
        for item, rui in rank:
            # 如果推荐的商品在该用户的书签字典中，说明推荐对了，则hit+1
            if item in items:
                hit = hit + 1
        # len(items) 实际打过标签的物品数
        h_recall += len(items)
        h_precision += N
    # 返回精确率 和 召回率
    prec = hit / (h_precision * 1.0)
    rec = hit / (h_recall * 1.0)
    return prec, rec

In [30]:
%%time 
print("NormTagBased-1")
testRecommend()

NormTagBased-1
推荐结果评估
  N        精确率        召回率
  5       0.907%       0.388%
 10       0.638%       0.546%
 20       0.507%       0.868%
 40       0.356%       1.218%
 60       0.287%       1.476%
 80       0.255%       1.750%
100       0.241%       2.061%
CPU times: user 6min 5s, sys: 1.45 s, total: 6min 6s
Wall time: 6min 6s


## 4.4 TagBased-TFIDF算法推荐

用户u对商品i的兴趣：
$$score(u, i) = \sum_{t}\frac{user\_tags[u, t]}{log(1 + tag\_users[t])} * tag\_items[t, i]$$
- 其中：
    - $user\_tags[u, t]$ 表示用户u使用过标签t的次数；    
    - $tag\_items[t, i]$ 表示商品i被打过标签t的次数；
    - $tag\_users[t]$ 表示标签t被多少个不同的用户使用。

### 4.4.1 自定义算法函数

In [31]:
def recommend_by_tfidf(user, N):
    tagged_items = user_items[user]
    recommend_items = defaultdict(int)
    for t, wut in user_tags[user].items():
        for i, wti in tag_items[t].items():
            if i in tagged_items:
                continue
            recommend_items[i] += (wut / np.log10(1 + len(tag_users[t]))) * wti
    return sorted(recommend_items.items(),
                 key=operator.itemgetter(1),
                 reverse=True)[:N]

In [32]:
%%time
recommend_by_tfidf(8, 3) 

CPU times: user 59.5 ms, sys: 998 µs, total: 60.5 ms
Wall time: 59.8 ms


[(1416, 28.267212282223298),
 (1526, 24.603389865469154),
 (4639, 22.533636146753715)]

### 4.4.2 评估TagBased-TFIDF算法

In [33]:
# 使用测试集，计算精确率和召回率
def precisionAndRecall(N):
    hit = 0
    h_recall = 0
    h_precision = 0
    # user用户，items是字典{书签ids: tag列表}
    for user, items in test_data.items():
        if user not in train_data:
            continue
        # 获取Top-N推荐列表
        rank = recommend_by_tfidf(user, N)
        # item商品(书签ID)，rui是兴趣分
        for item, rui in rank:
            # 如果推荐的商品在该用户的书签字典中，说明推荐对了，则hit+1
            if item in items:
                hit = hit + 1
        # len(items) 实际打过标签的物品数
        h_recall += len(items)
        h_precision += N
    # 返回精确率 和 召回率
    prec = hit / (h_precision * 1.0)
    rec = hit / (h_recall * 1.0)
    return prec, rec

In [34]:
%%time
print("TagBased-TFIDF算法:")
testRecommend()

TagBased-TFIDF算法:
推荐结果评估
  N        精确率        召回率
  5       1.008%       0.431%
 10       0.761%       0.652%
 20       0.549%       0.940%
 40       0.402%       1.376%
 60       0.328%       1.687%
 80       0.297%       2.033%
100       0.269%       2.306%
CPU times: user 19min 44s, sys: 2.13 s, total: 19min 47s
Wall time: 19min 47s


# 5 总结：

1. SimpleTagBased算法：
    - 用户对某个商品i的兴趣分score(u,i)是通过计算”用户u打标签t的次数“ 以及 ”商品i被打标签t的次数“ 之积 的和而得，需要对商品上的所有标签均计算乘积后，再求和； 
    - 该算法耗时较短，仅3min 7s。
2. NormTagBased-1 和 NormTagBased-2算法：
    - 两个算法是SimpleTagBased算法的归一化，两者分子都是SimpleTagBased算法的score(u,i)，不同的是它们的分母；
    - NormTagBased-1 的分母是 ”用户u打过多少个标签“ 以及 ”打过标签t的用户数“ 之积；
    - NormTagBased-2 的分母是 ”用户u打过多少个标签“ 以及 ”被打过标签t的商品数“ 之积；
    - NormTagBased-1 耗时: 6min 6s； NormTagBased-2耗时：6min 6s；两个算法的耗时相同；
    - NormTagBased-1 在该数据集上的精确率和召回率比 NormTagBased-2 都略高。
3. TagBased-TFIDF算法：
    - 借鉴TF-IDF的思想，分子仍是SimpleTagBased算法的score(u,i)，分母是 ”标签t被多少个用户使用“的对数，即：逆向文档概率IDF；
    - 耗时很长，需要19min 47s。
4. 3种算法的对比：
    - SimpleTagBased算法不愧是simple算法，耗时很短，精确率和召回率与NormTagBased的差距不大，但时间却短了一半；
    - TagBased-TFIDF耗时虽然最长，但精确率和召回率最高；
    - 在对单个用户id为8的用户进行推荐时:
        - SimpleTagBased的推荐结果:`[(1416, 61), (1526, 50), (4535, 47)]`
        - TagBased-TFIDF的推荐结果:`[(1416, 28.2672),(1526, 24.6033),(4639, 22.5336)]`
        - NormTagBased-1的推荐结果:`[(23702, 0.0108),(66188, 0.0107),(28750, 0.0104)]`
        - NormTagBased-2的推荐结果:`[(15, 0.0072),(23702, 0.0062),(66188, 0.0062)]`
        - 可见SimpleTagBased和TagBased-TFIDF的推荐结果相似度较大，而NormTagBased的2个算法和SimpleTagBased推荐的结果非常不同。

看起来，NormTagBased算法并不是一个好选择。如果我们需要追求推荐速度，可以用SimpleTagBased；如果我们想追求推荐的精确度则可以考虑TagBased-TFIDF算法。